In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping
import joblib

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Datasets/diabetes_dataset.csv")


In [ ]:
df.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [ ]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   gender               100000 non-null  object 
 1   age                  100000 non-null  float64
 2   hypertension         100000 non-null  int64  
 3   heart_disease        100000 non-null  int64  
 4   smoking_history      100000 non-null  object 
 5   bmi                  100000 non-null  float64
 6   HbA1c_level          100000 non-null  float64
 7   blood_glucose_level  100000 non-null  int64  
 8   diabetes             100000 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 6.9+ MB


,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,diabetes
count,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,41.885856,0.07485,0.039420,27.320767,5.527507,138.058060,0.085000
std,22.516840,0.26315,0.194593,6.636783,1.070672,40.708136,0.278883
min,0.080000,0.00000,0.000000,10.010000,3.500000,80.000000,0.000000
25%,24.000000,0.00000,0.000000,23.630000,4.800000,100.000000,0.000000
50%,43.000000,0.00000,0.000000,27.320000,5.800000,140.000000,0.000000
75%,60.000000,0.00000,0.000000,29.580000,6.200000,159.000000,0.000000
max,80.000000,1.00000,1.000000,95.690000,9.000000,300.000000,1.000000


In [ ]:
le = LabelEncoder()

df["gender"] = le.fit_transform(df["gender"])
df.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,0,80.0,0,1,never,25.19,6.6,140,0
1,0,54.0,0,0,No Info,27.32,6.6,80,0
2,1,28.0,0,0,never,27.32,5.7,158,0
3,0,36.0,0,0,current,23.45,5.0,155,0
4,1,76.0,1,1,current,20.14,4.8,155,0


In [ ]:
df["smoking_history"].value_counts()

,count
smoking_history,
No Info,35816
never,35095
former,9352
current,9286
not current,6447
ever,4004


In [ ]:
ohe = OneHotEncoder(drop='first',sparse_output=False)

smoking_encoded = ohe.fit_transform(df[['smoking_history']])

smoking_cols = ohe.get_feature_names_out(['smoking_history'])

smoking_df = pd.DataFrame(smoking_encoded, columns=smoking_cols)

df = pd.concat([df,smoking_df], axis=1)

df = df.drop("smoking_history", axis=1)
df.head()

,gender,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,diabetes,smoking_history_current,smoking_history_ever,smoking_history_former,smoking_history_never,smoking_history_not current
0,0,80.0,0,1,25.19,6.6,140,0,0.0,0.0,0.0,1.0,0.0
1,0,54.0,0,0,27.32,6.6,80,0,0.0,0.0,0.0,0.0,0.0
2,1,28.0,0,0,27.32,5.7,158,0,0.0,0.0,0.0,1.0,0.0
3,0,36.0,0,0,23.45,5.0,155,0,1.0,0.0,0.0,0.0,0.0
4,1,76.0,1,1,20.14,4.8,155,0,1.0,0.0,0.0,0.0,0.0


In [ ]:
x = df.drop("diabetes", axis=1)
y = df["diabetes"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,stratify=y,random_state=42)


In [ ]:
scaler = MinMaxScaler()
scaled_columns = ["age","bmi","HbA1c_level","blood_glucose_level"]
x_train[scaled_columns] = scaler.fit_transform(x_train[scaled_columns])
x_test[scaled_columns] = scaler.transform(x_test[scaled_columns])


In [ ]:
early_stop = EarlyStopping(monitor="val_loss", patience=15,restore_best_weights=True)


In [ ]:
# Build the Model

model = Sequential([
    Dense(128, activation="relu", input_shape=(x_train.shape[1],)),
    Dropout(0.4),
    Dense(64, activation = "relu"),
    Dropout(0.3),
    Dense(32, activation = "relu"),
    Dropout(0.2),
    Dense(16, activation = "relu"),
    Dropout(0.1),
    Dense(8, activation = "relu"),
    Dense(1, activation = "sigmoid")
    ])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Compile the Model

model.compile(
    optimizer = "adam",
    loss = "binary_crossentropy",
    metrics = ["accuracy"]
    )

In [ ]:
# Train the Model

history = model.fit(
    x_train,
    y_train,
    validation_split = 0.2,
    epochs = 400,
    batch_size = 128,
    callbacks = [early_stop],
    verbose = 1
    )

Epoch 1/400
500/500 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8675 - loss: 0.3891 - val_accuracy: 0.9549 - val_loss: 0.1239
Epoch 2/400
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9512 - loss: 0.1423 - val_accuracy: 0.9581 - val_loss: 0.1154
Epoch 3/400
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9557 - loss: 0.1292 - val_accuracy: 0.9597 - val_loss: 0.1139
Epoch 4/400
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9583 - loss: 0.1217 - val_accuracy: 0.9582 - val_loss: 0.1122
Epoch 5/400
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9595 - loss: 0.1178 - val_accuracy: 0.9603 - val_loss: 0.1093
Epoch 6/400
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9605 - loss: 0.1157 - val_accuracy: 0.9610 - val_loss: 0.1073
Epoch 7/400
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9614 - loss: 0.1138 - val_accuracy: 0.9612 - val_loss: 0.1071
Epoch 8/400
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9624 - loss: 0.1093 - val_accu

In [ ]:
loss, accuracy = model.evaluate(x_test,y_test)

print(f"The accuracy is {accuracy:.3f}")

625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9727 - loss: 0.0788
The accuracy is 0.972


In [ ]:
joblib.dump(model, "Diabetes_Predictor.pkl")

['Diabetes_Predictor.pkl']